### **01 - Incidence of MLTC**
#### **01B - Data preparation**


**Imports**

In [1]:
# required imports

# requires blank line after last import


**Parameter cell**

In [2]:
# parameter cell
incidence_schema = ""  # "mltc_incidence_outputs_v40_20230331"
segmentation_schema = ""  # "obh_segmentation_v40_20230331"
pipeline_schema = ""  # "pipeline_v40_20230331"

# optional, can be blank


In [3]:
# Set parameters in Spark configuration with 'param.' prefix (for use in SQL cells)
spark.conf.set("param.incidence_schema", incidence_schema)
spark.conf.set("param.segmentation_schema", segmentation_schema)
spark.conf.set("param.pipeline_schema", pipeline_schema)


---

#### **01B - Data preparation**

This section creates interim tables from the Segmentation Dataset (and corresponding pipeline tables) used for the analyses below.

**a - Extract all incidence changes that result in increases to the condition count**

This section uses the pipeline subsegment combinations table, which contains one row per person, with mutually exclusive and continuous date ranges for the periods of time they spend in each unqiue combination of subsegments.

**Note**
- With the config changes above, due to the removal of certain subsegments, a person may have multiple rows for the same combination of subsegments
- This can occur where the change between rows relates to a subsegment that is not included in the config
- To handle this, the newly calculated `condition_count` (which reflects the selected subsegment config) is compared across rows for the same person
- The logic looks for instances where the `condition_count` increases, based on the selected subsegments

<blockquote style="color: #333333; background-color: #FFBF00; padding: 10px; border-left: 6px solid #C48800;">
  <strong>💡 TODO (EBT):</strong> Update table name and column names when refreshing to versions post-v4.0_20230331.
</blockquote>


In [4]:
%%sql

CREATE    OR REPLACE TEMPORARY VIEW mm_incidence_transitions AS
SELECT    *
FROM      (
          SELECT    nhs_number,
                    person_id,
                    pssc.subsegment_combination_id,
                    ssc.new_subsegment_combination_name as subsegment_combination_name,
                    ssc.new_condition_count as condition_count,
                    pssc.start_date,
                    pssc.end_date,
                    LAG(pssc.subsegment_combination_id) OVER (
                    PARTITION BY nhs_number
                    ORDER BY  start_date,
                              end_date
                    ) AS previous_subsegment_combination_id,
                    LAG(ssc.new_subsegment_combination_name) OVER (
                    PARTITION BY nhs_number
                    ORDER BY  start_date,
                              end_date
                    ) AS previous_subsegment_combination_name,
                    LAG(ssc.new_condition_count) OVER (
                    PARTITION BY nhs_number
                    ORDER BY  start_date,
                              end_date
                    ) AS previous_condition_count
          FROM      ${param.pipeline_schema}.py_8_pipeline_subsegment_combination pssc
          INNER     JOIN ${param.incidence_schema}.dim_subsegment_combinations_config ssc ON ssc.old_subsegment_combination_id = pssc.subsegment_combination_id
          ) x
WHERE     condition_count > previous_condition_count -- NOTE - ignores backwards transitions


**b - Join to fact_model to get age and remove unregistered rows, and create transitions base table**

**Run time**: ~10 min run time on medium

**Output**: This is a person level table, containing one row for each incidence transition from a lower to higher condition count.

**Note**: 
- `start_date`s from the previous section are at a daily precision level, rather than monthly
- For this reason `LAST_DAY` is used to set these to the last day of the month, to allow for joining to the `fact_model` (which only contains a row per person for the end of each month)
- A pragmatic cut off is applied where previous condition count is >= 9 - these are grouped into a single category to avoid low volumes
- All analysis is restricted to adults aged 20+

<blockquote style="color: #D8000C; background-color: #FFD2D2; padding: 10px; border-left: 6px solid #D8000C;">
  <strong>⚠️ Warning:</strong> DROP TABLE is currently commented out, as this table does not need to be recreated each time the incidence analysis is run.
</blockquote>

In [5]:
%%sql

--DROP TABLE IF EXISTS ${param.incidence_schema}.mm_incidence_transitions_age

In [7]:
%%sql

CREATE    TABLE ${param.incidence_schema}.mm_incidence_transitions_age USING PARQUET AS
SELECT    f.date_id AS transition_month_date_id,
          d.financial_year,
          CASE
                    WHEN previous_condition_count > 9 THEN 9
                    ELSE previous_condition_count
          END AS previous_condition_count,
          CASE
                    WHEN condition_count > 10 THEN 10
                    ELSE condition_count
          END AS condition_count,
          p.person_id,
          p.pseudo_nhs_number,
          f.age_id AS age,
          CASE WHEN a.ten_year IN ('90-99','100-109','110-119') THEN '90+' ELSE a.ten_year END as age_band,
          p.gender_description,
          p.imd_quintile,
          e.census_2011_ethnic_group,
          t.previous_subsegment_combination_id,
          t.subsegment_combination_id
FROM      mm_incidence_transitions t
INNER     JOIN ${param.segmentation_schema}.dim_person p ON p.person_id = t.person_id
INNER     JOIN ${param.segmentation_schema}.dim_ethnicity e ON e.ethnicity_id = p.ethnicity_id
INNER     JOIN ${param.segmentation_schema}.dim_date d ON d.date = LAST_DAY(t.start_date)
INNER     JOIN ${param.segmentation_schema}.fact_model f ON f.person_id = p.person_id
AND       d.date_id = f.date_id
INNER     JOIN ${param.segmentation_schema}.dim_age a ON a.age_id = f.age_id
WHERE     f.gp_id IS NOT NULL
AND       f.age_id>=20


**c - Create denominator person time base table**

**Run time**: ~22 mins

**Output**: This is an aggregate table, containing one row for the person years associated with each combination of filters.



<blockquote style="color: #D8000C; background-color: #FFD2D2; padding: 10px; border-left: 6px solid #D8000C;">
  <strong>⚠️ Warning:</strong> DROP TABLE is currently commented out, as this table does not need to be recreated each time the incidence analysis is run.
</blockquote>

In [4]:
%%sql

--DROP TABLE IF EXISTS ${param.incidence_schema}.mm_incidence_person_time

In [5]:
%%sql 

CREATE TABLE ${param.incidence_schema}.mm_incidence_person_time USING PARQUET AS

SELECT    financial_year,
          CASE
                    WHEN new_condition_count > 10 THEN 10
                    ELSE new_condition_count
          END AS condition_count,
          CASE WHEN a.ten_year IN ('90-99','100-109','110-119') THEN '90+' ELSE a.ten_year END as age_band,
          p.gender_description,
          p.imd_quintile,
          e.census_2011_ethnic_group,
          COUNT(CASE WHEN d.last_day_of_financial_year = 1 THEN 1 ELSE NULL END) AS unique_people_fy_end,
          SUM(d.month_financial_year_fraction) AS person_years
FROM      ${param.segmentation_schema}.fact_model f
INNER     JOIN ${param.incidence_schema}.dim_subsegment_combinations_config ssc ON ssc.old_subsegment_combination_id = f.subsegment_combination_id
INNER     JOIN ${param.segmentation_schema}.dim_date d ON d.date_id = f.date_id
INNER     JOIN ${param.segmentation_schema}.dim_person p ON p.person_id = f.person_id
INNER     JOIN ${param.segmentation_schema}.dim_ethnicity e ON e.ethnicity_id = p.ethnicity_id
INNER     JOIN ${param.segmentation_schema}.dim_age a ON a.age_id = f.age_id
WHERE     f.gp_id IS NOT NULL
AND       f.age_id>=20
GROUP BY  financial_year,
          CASE
                    WHEN new_condition_count > 10 THEN 10
                    ELSE new_condition_count
          END,          
          CASE WHEN a.ten_year IN ('90-99','100-109','110-119') THEN '90+' ELSE a.ten_year END,
          p.gender_description,
          p.imd_quintile,
          e.census_2011_ethnic_group
